In [16]:
import numpy as np
import torch.nn as nn
import torch
import pandas
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as udata
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(0)
np.random.seed(0)

In [17]:
def accuarcy(dataloader): 
    correct = 0
    total = 0
    with torch.no_grad():
        for i, data in enumerate(dataloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            #print(inputs.size())
            #print("labels: ", labels, labels.size(0))
            outputs = cnn(inputs)
            #print(outputs.data)
            #might change
            softmax = nn.Softmax(dim=1)
            probability = softmax(outputs.data)
            _, predicted = torch.max(probability, 1)
            #print("pred: ", predicted)
            total += labels.size(0)
            #print("total: ", total)
            correct += (predicted == labels).sum().item()
            #print("correct: ", correct)
            if total % 100 == 98:
                print(total)
    #print(total)    
    #print('Accuracy of the network on the train data: %d %%' % (
        #100 * correct / total))
    return 100 * correct / total

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5)
        self.fc1 = nn.Linear(32 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

In [20]:
def train(cnn, trainloder, testloader, num, epsilon=0.003, filename = None):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(cnn.parameters(), lr= 0.001)
    pre_epoch = float("inf")
    for epoch in range(num):
        running_loss = 0.0
        total = 0 
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            #print(inputs.size())
            optimizer.zero_grad()
            outputs = cnn(inputs)
            #print(inputs.size(), outputs.size())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            total += 1
        test_accuarcy = accuarcy(testloader)
        train_accuarcy = accuarcy(trainloader)
        epoch_loss = running_loss / total
        print('epoch [%d] loss: %.3f' % (epoch + 1, epoch_loss), train_accuarcy, test_accuarcy)
        if filename != None:
            f = open(filename, 'a')
            f.write('epoch {} loss: {}, train_accuracy: {}, test_accuracy: {} \n'.format(epoch+1, epoch_loss, train_accuarcy, test_accuarcy))
        pre_epoch = epoch_loss
        
    print('Finished Training')
    if filename != None:
        f.close()

In [21]:
def train_model(use_singan, dir_name):
    #load train set
    trainpath = "cifar10-2classes-trainset"
    transformer = transforms.Compose(
        [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    if use_singan:
        #trainpath = "singan-train"
        trainset1 = datasets.ImageFolder(root=trainpath, transform=transformer)
        trainset2 = datasets.ImageFolder(root="singan-train", transform=transformer)
        trainset = udata.ConcatDataset([trainset1, trainset2])
    else:
        trainset = datasets.ImageFolder(root=trainpath, transform=transformer)
    
    #load test set
    testpath = "cifar10-2classes-testset"
    testset = datasets.ImageFolder(root=testpath, transform=transformer)
    
    #set up device and train/test loader
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    trainloader = udata.DataLoader(trainset, batch_size = 20, shuffle=True)
    testloader = udata.DataLoader(testset, batch_size = 20, shuffle=True)
    
    #Build CNN
    cnn = CNN()
    cnn.to(device)
    
    return cnn, trainloader, testloader 

In [25]:
# run train model 20 times, assign each experiment with a different random seed and save the output to a file

import time
import os
for experiment_num in range(20):
    t = int(time.time())
    
    dir_name = str(t)
    print(dir_name)
    #make directory
    os.system("mkdir result")
    os.system("mkdir ./result/{}".format(dir_name))
    
    #set random seed
    torch.manual_seed(t)
    np.random.seed(t)
    
    #train model without singan-generated images
    use_singan = False
    cnn, trainloader, testloader = train_model(use_singan, dir_name)
    
    
    #start training data and save the print out to a file
    
    if use_singan:
        filename = './result/{}/with_singan.txt'.format(dir_name)
    else:
        filename = './result/{}/without_singan.txt'.format(dir_name)
    train(cnn, trainloader, testloader, 30, filename = filename)
    
    
    #set random seed
    torch.manual_seed(t)
    np.random.seed(t)
    
    #train model with singan-generated images
    use_singan = True
    cnn, trainloader, testloader  = train_model(use_singan, dir_name)
    
    #start training data and save the print out to a file
    
    if use_singan:
        filename = './result/{}/with_singan.txt'.format(dir_name)
    else:
        filename = './result/{}/without_singan.txt'.format(dir_name)
    train(cnn, trainloader, testloader, 30, filename = filename)

1586387564
epoch [1] loss: 0.687 62.06543967280164 58.19095477386934
epoch [2] loss: 0.657 65.439672801636 61.256281407035175
epoch [3] loss: 0.633 68.40490797546012 64.87437185929649
epoch [4] loss: 0.601 70.44989775051124 65.52763819095478
epoch [5] loss: 0.563 73.92638036809817 65.87939698492463
epoch [6] loss: 0.535 75.20449897750511 65.52763819095478
epoch [7] loss: 0.490 81.95296523517382 66.03015075376885
epoch [8] loss: 0.430 81.0838445807771 66.83417085427136
epoch [9] loss: 0.389 87.88343558282209 67.98994974874371
epoch [10] loss: 0.319 89.2638036809816 68.09045226130654
epoch [11] loss: 0.242 93.9161554192229 67.8391959798995
epoch [12] loss: 0.174 95.91002044989776 66.13065326633166
epoch [13] loss: 0.135 95.5521472392638 66.48241206030151
epoch [14] loss: 0.124 97.6482617586912 65.32663316582915
epoch [15] loss: 0.059 99.2842535787321 66.98492462311557
epoch [16] loss: 0.063 98.46625766871166 66.48241206030151
epoch [17] loss: 0.079 99.13087934560328 66.88442211055276
epo